In [ ]:
import pandas as pd
from numpy import dtype
from ludwig.api import LudwigModel
from utils import get_training_data_set, get_model_definition, generate_base_model_definition
from new_api import (add_entity, create_new_entity_set, add_aggregation_features, get_training_df, get_prediction_df, 
add_entity_df, list_features)


In [1]:
!ls

,                         df_example.py             table_example.py
Dataframe API.ipynb       df_example2.py            table_example2.py
Demo 1.ipynb              feature_store.py          test1.py
Demo 2.ipynb              model creation demo.ipynb test2.py
Dockerfile                model_definition.yaml     titanic.csv
README.md                 new_api.py                utils.py
Titanic Demo.ipynb        requirements.txt          venv
__pycache__               sql-scripts


In [ ]:
generate_base_model_definition(df, target='Survived')

In [ ]:
%load model_definition.yaml
input_features:
- name: Id
  type: numerical
- name: Class
  type: numerical
- name: Name
  type: category
- name: Sex
  type: category
- name: Age
  type: numerical
- name: SibSp
  type: numerical
- name: Parch
  type: numerical
- name: Ticket
  type: category
- name: Fare
  type: numerical
- name: Cabin
  type: category
- name: Embarked
  type: category
output_features:
- name: Survived
  type: binary


## Iterate on model in cell below.  Results are written to a new directory every time

In [32]:
%%writefile model_definition.yaml
input_features:
- name: Class
  type: numerical
- name: Sex
  type: category
- name: Age
  type: numerical
  preprocessing:
          missing_value_strategy: fill_with_mean
          normalization: zscore
- name: SibSp
  type: numerical
- name: Parch
  type: numerical
- name: Fare
  type: numerical
- name: Cabin
  type: category
output_features:
- name: Survived
  type: binary
  num_fc_layers: 2
  fc_size: 128
training:
    batch_size: 128
    epochs: 200


Overwriting model_definition.yaml


In [29]:
ludwig_model = LudwigModel({}, model_definition_file='model_definition.yaml')
train_status = ludwig_model.train(data_df=df)

In [33]:
max(train_status['validation']['combined']['accuracy'])

0.8024691358024691

In [34]:
!ls results

api_experiment_run_0  api_experiment_run_13 api_experiment_run_6
api_experiment_run_1  api_experiment_run_2  api_experiment_run_7
api_experiment_run_10 api_experiment_run_3  api_experiment_run_8
api_experiment_run_11 api_experiment_run_4  api_experiment_run_9
api_experiment_run_12 api_experiment_run_5


In [ ]:
deploy_model_to_fastscore(ludwig_model)
#saves model to tmp directory (or locates experiement in results directory) and package as tar file
#generates a model conformance python file (same every time?) that just unpacks tar file
#using input features and output features generate schema definition file
#upload everything to fastscore using python sdk
#model_definition,yaml gets tagged and stored....now we can automate re-training model on new data and we can guarantee that model can be recreated.
